# **Project Name**    - Diabetes Prediction





##### **Project Type**    - EDA/Regression/Classification/Unsupervised
Made By- Vaibhav Verma

# **Project Summary -**

Diabetes Prediction using Machine Learning

Diabetes, is a group of metabolic disorders in which there are high blood sugar levels over a prolonged period. Symptoms of high blood sugar include frequent urination, increased thirst, and increased hunger. If left untreated, diabetes can cause many complications. Acute complications can include diabetic ketoacidosis, hyperosmolar hyperglycemic state, or death. Serious long-term complications include cardiovascular disease, stroke, chronic kidney disease, foot ulcers, and damage to the eyes.

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import statsmodels.api as sm
from collections import Counter
import os
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.simplefilter(action = "ignore")

### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import files
uploaded =files.upload()

### Dataset First View

In [ ]:
# Dataset First Look
df = pd.read_csv('/content/diabetes.csv')
df.head()

In [ ]:
# df= pd.read_csv('/content/diabetes.csv')
df.shape

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
num_rows, num_cols = df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

### Dataset Information

In [ ]:
# Dataset Info
df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
duplicate_rows = df[df.duplicated()]
print(duplicate_rows)
num_duplicate_rows = len(duplicate_rows)
print(num_duplicate_rows)

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
missing_values_count = df.isnull().sum()
print(missing_values_count)


## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
df["Age"].hist(edgecolor = "black");

##### 1. Why did you pick the specific chart?

Histograms provide a clear visual representation of the distribution of a continuous variable. They show the frequency or density of data within predefined intervals (bins), making it easy to identify patterns, trends, and outliers in the data.

#### Chart - 2

In [ ]:
# Chart - 2 visualization code
counts = df['Outcome'].value_counts()

plt.figure(figsize=(9, 9))  # Set the size of the pie chart
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=140)
plt.title('target')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
import seaborn as sns
sns.boxplot(x = df["Insulin"]);

#### Chart - 4

In [ ]:
# Chart - 4 visualization code
# Exploring Pregnancy and target variables together

plt.figure(figsize = (10, 8))

# Plotting density function graph of the pregnancies and the target variable
kde = sns.kdeplot(df["Pregnancies"][df["Outcome"] == 1], color = "Red", shade = True)
kde = sns.kdeplot(df["Pregnancies"][df["Outcome"] == 0], ax = kde, color = "Blue", shade= True)
kde.set_xlabel("Pregnancies")
kde.set_ylabel("Density")
kde.legend(["Positive Result", "Negative Result"])

#### Chart - 5

In [ ]:
# Chart - 5 visualization code
# Exploring the Glucose and the Target variables together
plt.figure(figsize = (10, 8))
sns.violinplot(data = df, x = "Outcome", y = "Glucose",
               split = True, inner = "quart", linewidth = 2)

#### Chart - 6

In [ ]:
# Chart - 6 visualization code
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Pregnancies', hue='Age')
plt.title('Distribution of regnancy rate')
plt.xlabel('pregnancies')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

#### Chart - 7

In [ ]:
# Chart - 7 visualization code
sns.kdeplot(data=df, x="SkinThickness",hue="Age",fill=True,alpha=0.8, linewidth=1.5)

#### Chart - 8 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code
f, ax = plt.subplots(figsize= [15,10])
sns.heatmap(df.corr(), annot=True, fmt=".2f", ax=ax, cmap = "magma" )
ax.set_title("Correlation Matrix", fontsize=15)
plt.show()

#### Chart - 9 - Pair Plot

In [ ]:
# Pair Plot visualization code
sns.pairplot(df)
plt.show()

#### Chart - 10


In [ ]:
sns.scatterplot(x= "Insulin", y ="Age", data = df)

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
df.head()

In [ ]:
# Now, we can look at where are missing values
df.isnull().sum()

In [ ]:
# Have been visualized using the missingno library for the visualization of missing observations.
# Plotting
import missingno as msno
msno.bar(df);

In [ ]:
# The missing values ​​will be filled with the median values ​​of each variable.
def median_target(var):
    temp = df[df[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp
    # The values to be given for incomplete observations are given the median value of people who are not sick and the median values of people who are sick.
columns = df.columns
columns = columns.drop("Outcome")
for i in columns:
    median_target(i)
    df.loc[(df['Outcome'] == 0 ) & (df[i].isnull()), i] = median_target(i)[i][0]
    df.loc[(df['Outcome'] == 1 ) & (df[i].isnull()), i] = median_target(i)[i][1]
    df.head()

In [ ]:
# Missing values were filled.
df.isnull().sum()

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments
# In the data set, there were asked whether there were any outlier observations compared to the 25% and 75% quarters.
# It was found to be an outlier observation.
for feature in df:

    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3-Q1
    lower = Q1- 1.5*IQR
    upper = Q3 + 1.5*IQR

    if df[(df[feature] > upper)].any(axis=None):
        print(feature,"yes")
    else:
        print(feature, "no")

In [ ]:
# The process of visualizing the Insulin variable with boxplot method was done. We find the outlier observations on the chart.
import seaborn as sns
sns.boxplot(x = df["Insulin"]);

In [ ]:
# We determine outliers between all variables with the LOF method
from sklearn.neighbors import LocalOutlierFactor
lof =LocalOutlierFactor(n_neighbors= 10)
lof.fit_predict(df)


In [ ]:
df_scores = lof.negative_outlier_factor_
np.sort(df_scores)[0:30]

In [ ]:
#We choose the threshold value according to lof scores
threshold = np.sort(df_scores)[7]
threshold

In [ ]:
#We delete those that are higher than the threshold
outlier = df_scores > threshold
df = df[outlier]

In [ ]:
df.shape

### 3. Categorical Encoding

In [ ]:
# According to BMI, some ranges were determined and categorical variables were assigned.
NewBMI = pd.Series(["Underweight", "Normal", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"], dtype = "category")
df["NewBMI"] = NewBMI
df.loc[df["BMI"] < 18.5, "NewBMI"] = NewBMI[0]
df.loc[(df["BMI"] > 18.5) & (df["BMI"] <= 24.9), "NewBMI"] = NewBMI[1]
df.loc[(df["BMI"] > 24.9) & (df["BMI"] <= 29.9), "NewBMI"] = NewBMI[2]
df.loc[(df["BMI"] > 29.9) & (df["BMI"] <= 34.9), "NewBMI"] = NewBMI[3]
df.loc[(df["BMI"] > 34.9) & (df["BMI"] <= 39.9), "NewBMI"] = NewBMI[4]
df.loc[df["BMI"] > 39.9 ,"NewBMI"] = NewBMI[5]

In [ ]:
# A categorical variable creation process is performed according to the insulin value.
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"

In [ ]:
# The operation performed was added to the dataframe.
df = df.assign(NewInsulinScore=df.apply(set_insulin, axis=1))

df.head()

In [ ]:

# Some intervals were determined according to the glucose variable and these were assigned categorical variables.
NewGlucose = pd.Series(["Low", "Normal", "Overweight", "Secret", "High"], dtype = "category")
df["NewGlucose"] = NewGlucose
df.loc[df["Glucose"] <= 70, "NewGlucose"] = NewGlucose[0]
df.loc[(df["Glucose"] > 70) & (df["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]
df.loc[(df["Glucose"] > 99) & (df["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]
df.loc[df["Glucose"] > 126 ,"NewGlucose"] = NewGlucose[3]

In [ ]:
# Here, by making One Hot Encoding transformation, categorical variables were converted into numerical values. It is also protected from the Dummy variable trap.
df = pd.get_dummies(df, columns =["NewBMI","NewInsulinScore", "NewGlucose"], drop_first = True)

In [ ]:
df.head()

In [ ]:
categorical_df = df[['NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret']]

In [ ]:
categorical_df.head()

In [ ]:
y = df["Outcome"]
X = df.drop(["Outcome",'NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret'], axis = 1)
cols = X.columns
index = X.index

In [ ]:
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(X)
X = transformer.transform(X)
X = pd.DataFrame(X, columns = cols, index = index)

In [ ]:
X.head()

In [ ]:
X = pd.concat([X,categorical_df], axis = 1)

In [ ]:
y.head()

#### What all categorical encoding techniques have you used & why did you use those techniques?

1.pd.get_dummies() is a function in the pandas library in Python that is used to convert categorical variables into dummy/indicator variables. This process is also known as one-hot encoding.

2.In One-hot encoding, a categorical variable is transformed into a binary matrix where each column represents a category and each row represents an observation. Each cell in the matrix is filled with a binary value of 1 or 0 indicating the presence or absence of a particular category in that observation.

### 4. Data Scaling

In [ ]:
df.head()

In [ ]:
# Transforming the data into quartiles
from sklearn.preprocessing import QuantileTransformer
quartile  = QuantileTransformer()
X = quartile.fit_transform(df)
dataset = quartile.transform(X)
dataset = pd.DataFrame(X)
dataset.columns =['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome','NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret']
# Showing the top 5 rows of the transformed dataset
dataset.head()


### 5. Data Splitting

In [ ]:
# Splitting the dependent and independent features
X = df.drop(["Outcome"], axis = 1)
Y = df["Outcome"]

# Splitting the dataset into the training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.40, random_state = 10)

# Printing the size of the training and testing dataset
print("The size of the training dataset: ", X_train.size)
print("The size of the testing dataset: ", X_test.size)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

##### What data splitting ratio have you used and why?

the 40 to 60 ratio


## ***7. ML Model Implementation***

### ML Model - 1

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

The machine learning model used above is a Decision Tree Classifier. Decision trees are a type of supervised learning algorithm used for classification and regression tasks. In a decision tree, the data is split into subsets based on the values of features, with the aim of creating a tree-like model of decisions. At each node of the tree, the algorithm selects the feature that best splits the data into classes (in the case of classification) or minimizes the variance (in the case of regression).

In [ ]:
# Visualizing evaluation Metric Score chart
clf = DecisionTreeClassifier(random_state=100)

# Train the model
clf.fit(X_train, Y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(Y_test, y_pred))

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model


# Define the parameter grid for hyperparameter tuning
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=100)

# Perform Grid Search Cross-Validation
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, Y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Initialize Decision Tree Classifier with best parameters
best_clf = DecisionTreeClassifier(**best_params)

# Evaluate using Cross-Validation
cv_scores = cross_val_score(best_clf, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())


##### Which hyperparameter optimization technique have you used and why?

I used Grid Search Cross-Validation for hyperparameter optimization.

Grid Search Cross-Validation is a technique used to tune the hyperparameters of a machine learning model by exhaustively searching through a specified grid of hyperparameter values. It works by evaluating the model performance for each combination of hyperparameters using cross-validation and selecting the combination that yields the best performance according to a specified evaluation metric.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

In this scenario, after hyperparameter tuning, the model's accuracy increased from 0.85 to a mean cross-validation score of 0.86, indicating an improvement in performance. The best parameters found through grid search suggest that certain hyperparameter settings were more effective in optimizing the model's performance for the given dataset.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

The machine learning model used in the provided example is a k-Nearest Neighbors (kNN) classifier for predicting diabetes.

In [ ]:
# Visualizing evaluation Metric Score chart
# Initialize the kNN classifier
k = 5  # Choose the value of k
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(X_train, Y_train)

# Predictions
y_pred = knn_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(Y_test, y_pred))

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# Define a range of k values to search
param_grid = {'n_neighbors': range(1, 21)}  # Search for k values from 1 to 20

# Initialize the kNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(knn_classifier, param_grid, cv=5, scoring='accuracy')

# Perform grid search to find the best k value
grid_search.fit(X_train, Y_train)

# Print the best k value and corresponding accuracy
print("Best k value:", grid_search.best_params_['n_neighbors'])
print("Best Accuracy:", grid_search.best_score_)

# Re-train the classifier with the best k value
best_k = grid_search.best_params_['n_neighbors']
best_knn_classifier = KNeighborsClassifier(n_neighbors=best_k)
best_knn_classifier.fit(X_train, Y_train)

# Evaluate the classifier using cross-validation
cv_scores = cross_val_score(best_knn_classifier, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

# Predictions
y_pred = best_knn_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(Y_test, y_pred)
print("Test Set Accuracy:", accuracy)

##### Which hyperparameter optimization technique have you used and why?

 I used Grid Search for hyperparameter optimization.
 it is suitable for relatively small parameter grids and datasets, making it a good choice for this scenario. Additionally, for more efficient search in larger spaces, techniques like Random Search or Bayesian Optimization can be considered.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

If you run the provided code on your dataset and compare the performance of the model before and after hyperparameter tuning, you should be able to observe any improvements in terms of accuracy or other evaluation metrics.The accuracy was 0.71 before hyperparameter, after tuning the acc. went to 0.79.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Each metric provides valuable insights into different aspects of the model's performance, guiding businesses in making informed decisions and optimizing outcomes.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model
from sklearn.linear_model import LogisticRegression
logistic_reg = LogisticRegression(random_state=42)

# Train the model
X = df.drop('Outcome', axis=1)  # Features
Y = df['Outcome']  # Target variable

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

logistic_reg.fit(X_train, Y_train)

# Predict on the test set
y_pred = logistic_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print(classification_report(Y_test, y_pred))

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

Logistic regression is a classification algorithm used to model the probability of a binary outcome (in this case, whether an individual has diabetes or not) based on one or more predictor variables (features such as glucose levels, BMI, age, etc.).
Model Training: The logistic regression model is trained on a labeled dataset containing features and corresponding binary labels indicating the presence or absence of diabetes.
Output Interpretation: Logistic regression outputs probabilities between 0 and 1, representing the likelihood of belonging to the positive class (diabetic). These probabilities can be thresholded to make binary predictions.

In [ ]:
# Visualizing evaluation Metric Score chart
# Plotting the actual vs predicted values
# Generate predictions
y_pred = logistic_reg.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(Y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(Y_test, y_pred))

# Accuracy
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Define hyperparameter grid
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'C': uniform(loc=0, scale=4),  # Regularization parameter
    'penalty': ['l1', 'l2']        # Penalty (L1 or L2 regularization)
}

# Initialize logistic regression classifier
log_reg = LogisticRegression(max_iter=1000)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(log_reg, param_distributions=param_grid, n_iter=100, cv=5, scoring='accuracy', random_state=42)

# Perform random search to find the best hyperparameters
random_search.fit(X_train, Y_train)

# Print the best hyperparameters
print("Best hyperparameters:", random_search.best_params_)

# Re-train the model using the best hyperparameters
best_log_reg = random_search.best_estimator_

# Evaluate the model using cross-validation
cv_scores = cross_val_score(best_log_reg, X_train, Y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

# Predictions
y_pred = best_log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print("Test Set Accuracy:", accuracy)

##### Which hyperparameter optimization technique have you used and why?

Overall, Randomized Search Cross-Validation strikes a balance between exhaustively searching the entire hyperparameter space (as done in Grid Search) and sampling random points to efficiently find good hyperparameter configurations. It's particularly useful when there are many hyperparameters to tune or when computational resources are limited.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Before Hyperparameter Tuning:

Accuracy: 0.74
Precision: 0.68
Recall: 0.72
F1-score: 0.70
After Hyperparameter Tuning:

Accuracy: 0.76
Precision: 0.72
Recall: 0.76
F1-score: 0.74
In this hypothetical scenario, we observe improvements in all evaluation metrics after hyperparameter tuning. The accuracy has increased from 0.74 to 0.75, indicating that a higher proportion of instances are correctly classified.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

When considering evaluation metrics for a positive business impact, it's important to select metrics that directly align with the business objectives and goals. For a diabetes prediction task, where the goal might be to identify individuals at risk of diabetes, the following evaluation metrics could be considered:

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

The logistic regression model would be my preferred choice as the final prediction model for diabetes. It strikes a balance between performance, interpretability, and robustness, making it suitable for practical deployment in healthcare settings.






### 3. Explain the model which you have used and the feature importance using any model explainability tool?

In [ ]:
pip install shap


In [ ]:
import shap

# Initialize SHAP explainer with the logistic regression model
explainer = shap.Explainer(best_log_reg, X_train)

# Calculate SHAP values
shap_values = explainer.shap_values(X_train)

# Plot feature importance
shap.summary_plot(shap_values, X_train, plot_type="bar", class_names=["Not Diabetic", "Diabetic"])


Let's delve into the logistic regression model used for diabetes prediction and explore feature importance using a popular model explainability tool called "SHAP" (SHapley Additive exPlanations).


The resulting plot will show the feature importance values, indicating which features have the most significant impact on the model's predictions for both the "Diabetic" and "Not Diabetic" classes. Features with higher SHAP values are considered more important in influencing the model's predictions.

# **Conclusion**

For diabetes prediction, where interpretability and simplicity are important considerations, logistic regression emerges as the preferred choice. Its ability to provide easily interpretable results, scalability, and well-established effectiveness in binary classification tasks make it a practical option for real-world deployment.

While decision trees and kNN classifiers have their strengths, such as interpretability and ability to capture complex relationships, they may not offer the same level of interpretability and generalization performance as logistic regression for this specific task. Moreover, decision trees are prone to overfitting, and kNN can be computationally expensive.

Therefore, based on the specific requirements and priorities of the diabetes prediction task, logistic regression stands out as the most suitable model choice.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***